In [2]:
%%html
<style>.text_cell .rendered_html * {direction: rtl; text-align: right;}</style>

# پروژه ۱:
در این پروژه به بررسی چند الگوریتم جستوجوی آگاهانه و ناآگاهانه میپردازیم و الگوریتم های جستوجوی ناآگاهانه BFS و IDS و آگاهانه ی A* را پیاده سازی کرده ایم.

### دریافت ورودی ها:

In [22]:
import time

In [143]:
f = open("Tests/4.txt", "r")
n, m = list(map(int,f.readline().split()))
s_pos = tuple(map(int,f.readline().split()))
f_pos = tuple(map(int,f.readline().split()))
c = int(f.readline())
k = int(f.readline())
# print(k)
orb_pos = dict()
for i in range(k):
    pos = tuple(map(int,f.readline().split())) #(y1,x1,y2,x2)
    orb_pos[i] = pos
    
map_ = list()
for line in f:
    line = line.replace(" ", "")
    map_.append(line[:-1])
border_x = len(map_[0])
border_y = len(map_)

print ( border_x, border_y)
    
f.close()

30 30



در بالا عملیات گرفتن ورودی ها انجام شده است.

### تعریف کلاس State:

In [144]:
class State:
    def __init__(self, x, y, cap, car_orbs, del_orbs):
        self.x = x
        self.y = y
        self.cap = cap
        self.car_orbs = car_orbs
        self.del_orbs = del_orbs
    
    def __hash__(self):
        return hash((self.x, self.y, self.cap, self.car_orbs, self.del_orbs))
    def __eq__(self, other):
        return (self.x, self.y, self.cap, self.car_orbs, self.del_orbs) == (other.x, other.y, other.cap, other.car_orbs, other.del_orbs)
#         return (self.x, self.y) == (other.x, other.y)
        
    def __ne__(self, other):
        return not(self == other)
    
    def __lt__(self, other):
        return self.cap < other.cap


در بالا کلاس State برای معرفی هر حالت در طول جستوجوی ما تعریف شده است.

هر حالت دارای مختصات x,y میباشد

cap عبارت است از ظرفیت باقی مانده ایندی برای برداشتن گوی های بعدی است

car_orbs بصورت tuple ای از ۰ و ۱ است که اندیس iام این آرایه مشخص میکند که آیا گوی iام همراه ایندی است یا خیر.

del_orbs نیز همانند car_orbs تاپلی از ۰ و ۱ است که مشخص میکند کدام گوی ها به مقصد رسیده اند.

توابع hash و ne و eq  معرفی شده اند تا از این کلاس بتوان بعنوان کلید دیکشنری استفاده کرد.

تابع lt برای رفع ارور در heapify کردن در بخش A* استفاده شده است.

### تعریف شرایط آغاز و خاتمه:

In [145]:
init_car_orbs = tuple([0 for i in range(k)])
init_del_orbs = tuple([0 for i in range(k)])

fin_car_orbs = tuple([0 for i in range(k)])
fin_del_orbs = tuple([1 for i in range(k)])

init_state = State(s_pos[1], s_pos[0], c, init_car_orbs, init_del_orbs)
fin_state = State(f_pos[1], f_pos[0], c, fin_car_orbs, fin_del_orbs)

dirs = ( (0,1), (1,0), (-1, 0), (0,-1)) #(x,y)
print(init_car_orbs)

()


در بالا شرط آغاز یعنی initial state به این صورت تعریف شده است که در مختصات آغاز, با ظرفیت c و به ازای هر گوی مقادیر carry و delivered صفر میباشند.

شرط پایان یعنی finish state نیز با مختصات خانه پایان, ظرفیت c, گوی های carry صفر و گوی های delivered یک تعریف شده است.

تاپل dirs برای مشخص کردن جهتهای قابل جستوجو بکار رفته است.


## الگوریتم A*:

In [190]:
def h1(s):
    inf = 999999
    for_orb = [inf]*k
    for i in range(k):
        if s.car_orbs[i] == 1:
            for_orb[i] = abs(s.x - orb_pos[i][3]) + abs(s.y - orb_pos[i][2])
            
        if s.car_orbs[i] == 0 and s.del_orbs[i] == 0 and s.cap > 0:
            for_orb[i] = abs(s.x - orb_pos[i][1]) + abs(s.y - orb_pos[i][0]) + 1
    
    sum_of_del = 0
    for deliv in s.del_orbs:
        sum_of_del += deliv
    all_orbs_delivered = (sum_of_del == k)
    
    if all_orbs_delivered:
        h = abs(s.x - fin_state.x) + abs(s.y - fin_state.y)
    else:
        h = min(for_orb)
    return h

h2_table = {}
def h2(s):
    
    inf = 999999
    for_orb = [inf]*k
    
    new_car_orbs = list(s.car_orbs)
    new_del_orbs = list(s.del_orbs)
    
    for i in range(k):
        #deliver orb
        if s.car_orbs[i] == 1:
            new_car_orbs[i] = 0
            new_del_orbs[i] = 1
            new_s = State(orb_pos[i][3], orb_pos[i][2], s.cap + 1, tuple(new_car_orbs), tuple(new_del_orbs))
            new_car_orbs[i] = 1
            new_del_orbs[i] = 0
            
            for_orb[i] = abs(s.x - orb_pos[i][3]) + abs(s.y - orb_pos[i][2])
            if new_s in h2_table:
                for_orb[i] += h2_table[new_s]
            else:
                for_orb[i] += h2(new_s)
            
        #get orb
        if s.car_orbs[i] == 0 and s.del_orbs[i] == 0 and s.cap > 0:
            new_car_orbs[i] = 1
            new_s = State(orb_pos[i][1], orb_pos[i][0], s.cap - 1, tuple(new_car_orbs), tuple(new_del_orbs))
            new_car_orbs[i] = 0
            
            for_orb[i] = abs(s.x - orb_pos[i][1]) + abs(s.y - orb_pos[i][0]) + 1
            if new_s in h2_table:
                for_orb[i] += h2_table[new_s]
            else:
                for_orb[i] += h2(new_s)
    
    
#     print(for_orb)
    sum_of_del = 0
    for deliv in s.del_orbs:
        sum_of_del += deliv
    all_orbs_delivered = (sum_of_del == k)
    
    if all_orbs_delivered:
        h = abs(s.x - fin_state.x) + abs(s.y - fin_state.y)
    else:
        h = min(for_orb)
        
    h2_table[s] = h 
    return h

alpha = 5
def heur(s):
    return h2(s) * alpha

tick = time.time()
print("init state heuristic is: ", heur(init_state))
from heapq import heapify, heappush, heappop

depth = dict()
explored = dict()
depth[init_state] = 0

q = [(depth[init_state] + heur(init_state),init_state)]
heapify(q)

count = 0
count2 = 0
while len(q) > 0:
    s = heappop(q)[1]
    explored[s] = 1
     
    #finish state
    if (s == fin_state):
        print('path found with ', depth[s], ' steps')
        print("seperate states : " , count)
        print("all states : " , count2)
        break
    
    #find orb state
    for i in range(k):
        if s.del_orbs[i] == 0 and s.car_orbs[i] == 0 and s.cap > 0 and s.x == orb_pos[i][1] and s.y == orb_pos[i][0]:
            s_new = State(s.x, s.y, s.cap, s.car_orbs, s.del_orbs)
            
            new_car_orbs = list(s_new.car_orbs)
            new_car_orbs[i] = 1
            
            s_new.car_orbs = tuple(new_car_orbs)
            
            s_new.cap -= 1
            depth[s_new] = depth[s] + 1
            if s_new not in explored:
                count += 1
            count2 += 1
            explored[s_new] = 1
            heappush(q, (depth[s_new] + heur(s_new), s_new))
    
    #explore
    for dir_ in dirs:
        if s.y+dir_[1] >= 0 and s.y + dir_[1] < border_y and s.x + dir_[0] >= 0 and s.x + dir_[0] < border_x and map_[s.y + dir_[1]][s.x + dir_[0]] == '-':
            s_new = State(s.x, s.y, s.cap, s.car_orbs, s.del_orbs)
            s_new.x = s.x + dir_[0]
            s_new.y = s.y + dir_[1]
            #deliver orb state
            for i in range(k):
                if s_new.x == orb_pos[i][3] and s_new.y == orb_pos[i][2] and s_new.del_orbs[i] == 0 and s_new.car_orbs[i] == 1:
                    new_car_orbs = list(s_new.car_orbs)
                    new_car_orbs[i] = 0

                    new_del_orbs = list(s_new.del_orbs)
                    new_del_orbs[i] = 1

                    s_new.car_orbs = tuple(new_car_orbs)
                    s_new.del_orbs = tuple(new_del_orbs)

                    s_new.cap += 1
            #___________________
            if s_new not in explored or depth[s_new] > depth[s]+1:
                if s_new not in explored:
                    count += 1
                count2 += 1
                explored[s_new] = 1
                depth[s_new] = depth[s] + 1
                heappush(q, (depth[s_new] + heur(s_new), s_new))
tock = time.time()
print(tock - tick)


init state heuristic is:  280
path found with  92  steps
seperate states :  428
all states :  745
0.020194053649902344


الگوریتم A* بهینه ترین الگوریتم بنظر میرسد.

در اینجا استیت ها را در heap ذخیره میکنیم تا در کمترین زمان بتوانیم استیت با
f(n) 
کمتر را پیدا کنیم.

۲
heuristic
معرفی شده است:
h1 و h2.

هوریستیک یک consistent است اما admissible نیست.
این هوریستیک مقدار کمترین حرکت برای رسیدن به یک task انجام نشده را پیدا میکند.
مثلا نزدیکترین گوی قابل حمل کردن یا نزدیکترین مقصد برای گوی حمل شده یا در صورت اتمام رساندن گوی ها,
فاصله تا مختصات goal را به ما میدهد.
توجه شود که همه ی فاصله ها فاصله ی Manhattan مختصات آنها میباشد.

هوریستیک ۲ admissible است.
این هوریستیک همانند مدل TSP, کوتاهترین مسیر برای برداشتن و ارسال همه ی گوی ها و سپس رفتن به مختصات goal را پیدا میکند.
فاصله ها همان فاصله Manhattan میباشد.
از آنجایی محاسبه این هوریستیک هر دفعه زمان زیادی برعکس اولی لازم دارد, از یک دیکشنری برای ذخیره کردن هر حالت استفاده میکنیم.
(h2_table)

بطور کلی هوریستیک ۱ ساده تر است و زمان کمتری برای محاسبه آن لازم است اما admissible نیست
چرا که شرط admissible بودن را ندارد.
اما 

تابع heur برای Weighted A* میباشد.

بقیه الگوریتم همانند قسمت DFS عه IDS پیاده سازی شده است.

## الگوریتم IDS:

In [157]:
from queue import LifoQueue
# from queue import Queue
maxDepth = 1000000

tick = time.time()

finished = False
count2 = 1
for limit in range(maxDepth):
    stack = LifoQueue()

    explored = dict()
    depth = dict()

    stack.put(init_state)
    depth[init_state] = 0
    count = 1
    while not stack.empty():
        s = stack.get()
        explored[s] = 1
            
        #finish state
        if (s == fin_state):
            print('path found with ', depth[s], ' steps')
            print("seperate states : " , count)
            print("all states : " , count2)
            finished = True
            print(count)
            break
        #limit reached
        if depth[s] >= limit:
            continue
        #find orb state
        for i in range(k):
            if s.del_orbs[i] == 0 and s.car_orbs[i] == 0 and s.cap > 0 and s.x == orb_pos[i][1] and s.y == orb_pos[i][0]:
                s_new = State(s.x, s.y, s.cap, s.car_orbs, s.del_orbs)

                new_car_orbs = list(s_new.car_orbs)
                new_car_orbs[i] = 1

                s_new.car_orbs = tuple(new_car_orbs)

                s_new.cap -= 1
                depth[s_new] = depth[s] + 1
                if s_new not in explored:
                    count += 1
                count2 += 1
                explored[s_new] = 1
                stack.put(s_new)

        #explore
        for dir_ in dirs:
            if s.y+dir_[1] >= 0 and s.y + dir_[1] < border_y and s.x + dir_[0] >= 0 and s.x + dir_[0] < border_x and map_[s.y + dir_[1]][s.x + dir_[0]] == '-':
                s_new = State(s.x, s.y, s.cap, s.car_orbs, s.del_orbs)
                s_new.x = s.x + dir_[0]
                s_new.y = s.y + dir_[1]
                #deliver orb state
                for i in range(k):
                    if s_new.x == orb_pos[i][3] and s_new.y == orb_pos[i][2] and s_new.del_orbs[i] == 0 and s_new.car_orbs[i] == 1:
                        new_car_orbs = list(s_new.car_orbs)
                        new_car_orbs[i] = 0

                        new_del_orbs = list(s_new.del_orbs)
                        new_del_orbs[i] = 1

                        s_new.car_orbs = tuple(new_car_orbs)
                        s_new.del_orbs = tuple(new_del_orbs)

                        s_new.cap += 1
                #___________________
                if s_new not in explored or depth[s_new] > depth[s]+1:
                    if s_new not in explored:
                        count += 1
                    count2 += 1
                    explored[s_new] = 1
                    depth[s_new] = depth[s] + 1
                    stack.put(s_new)
    if finished == True:
        break
#     print("limit is : ", limit)
#     print("count is : ", count)
#     print("____________________")
tock = time.time()
print("Time elapsed : ", tock - tick)

path found with  92  steps
seperate states :  316
all states :  87774
316
Time elapsed :  1.8572685718536377


در این الگوریتم بجای صف از استک استفاده شده است.
(LifoQueue)

در IDS برای optimal بودن میبایست از limit استفاده کرد که محدودیت عمق برای سرچ DFS را مشخص میکند.
از آنجایی که هزینه هر حرکت یک واحد است, limit هر سری با پیدا نشدن شرط پایان یک واحد اضافه میشود.

به ازای هر limit, یکبار بصورت کامل سرچ DFS را انجام میدهیم.
فرق سرچ DFS با BFS اینست که از آنجایی که ترتیب عمق در بازدید استیتها رعایت نمیشود, استیت مورد بازدید همیشه بهترین عمق را ندارد بنابراین باید چندین بار بازدید شود.

همچنین در IDS نباید استیت های با عمق limit گسترش داده شوند.

تنها مزیت IDS نسبت به بقیه, پیچیدگی فضایی بهتر آن است.

## الگوریتم BFS:

In [154]:
from queue import Queue
q = Queue()
explored = dict()
depth = dict()

q.put(init_state)
depth[init_state] = 0

count = 1

tick = time.time()
while not q.empty():
    s = q.get()
    explored[s] = 1

    #finish state
    if (s == fin_state):
        print('path found with ', depth[s], ' steps')
        print("seperate states = all states : " , count)
        break
    
    #find orb state
    for i in range(k):
        if s.del_orbs[i] == 0 and s.car_orbs[i] == 0 and s.cap > 0 and s.x == orb_pos[i][1] and s.y == orb_pos[i][0]:
            s_new = State(s.x, s.y, s.cap, s.car_orbs, s.del_orbs)
            
            new_car_orbs = list(s_new.car_orbs)
            new_car_orbs[i] = 1
            
            s_new.car_orbs = tuple(new_car_orbs)
            
            count += 1
            
            s_new.cap -= 1
            depth[s_new] = depth[s] + 1
            q.put(s_new)
    
    #explore
    for dir_ in dirs:
        if s.y+dir_[1] >= 0 and s.y + dir_[1] < border_y and s.x + dir_[0] >= 0 and s.x + dir_[0] < border_x and map_[s.y + dir_[1]][s.x + dir_[0]] == '-':
#             s_new = copy(s)
            s_new = State(s.x, s.y, s.cap, s.car_orbs, s.del_orbs)
            s_new.x = s.x + dir_[0]
            s_new.y = s.y + dir_[1]
            #deliver orb state
            for i in range(k):
                if s_new.x == orb_pos[i][3] and s_new.y == orb_pos[i][2] and s_new.del_orbs[i] == 0 and s_new.car_orbs[i] == 1:
                    new_car_orbs = list(s_new.car_orbs)
                    new_car_orbs[i] = 0

                    new_del_orbs = list(s_new.del_orbs)
                    new_del_orbs[i] = 1

                    s_new.car_orbs = tuple(new_car_orbs)
                    s_new.del_orbs = tuple(new_del_orbs)

                    s_new.cap += 1
            #___________________
            if s_new not in explored:
                count += 1
                explored[s_new] = 1
                depth[s_new] = depth[s] + 1
                q.put(s_new)
#             else:
#                 print("explored[s_new] : ", explored[s_new])
tock = time.time()
print(tock - tick)

path found with  92  steps
seperate states = all states :  457
0.02174854278564453


در الگوریتم BFS از صف برای بررسی استیت های مختلف استفاده شده است.

دیکشنری explored با کلید State مشخص میکند که آیا استیت موردنظر بازدید شده یا در صف قرار گرفته شده است.
explored کمک میکند تا از بازدید مجدد استیت های تکراری یا افتادن در لوپ جلوگیری شود.

دیکشنری depth با کلید استیت, تعداد حرکتهای لازم برای رسیدن از initial state تا استیت موردنظر را نشان میدهد.

count برای شمردن استیت های دیده شده است.

عملیات سرچ در یک while انجام میشود که با رسیدن به شرط پایان, از آن خارج میشود.
از آنجایی که صف FIFO است, همه ی استیت ها بترتیب صعودی depthهایشان بررسی میشوند بنابراین اولین جواب برای شرط پایان, بهترین جواب است.

شرط خاتمه در ابتدای while بررسی میشود.
(finish state)

در صورتی که به پایان نرسیده باشیم, باید استیت را گسترش دهیم.
ابتدا بررسی میکنیم که آیا در مختصات استیت گویی میتوان برداشت یا خیر
(find orb).
در اینصورت استیت جدید با یک حرکت بیشتر و با یک گوی همراه اضافه میشود که در صف برای بررسی قرار میگیرد.

در ادامه باید استیت را در صورت امکان به ۴ جهت بالا پایین چپ راست گسترش دهیم.
هنگام گسترش دادن باید در صورت امکان گوی های قابل رسیدن به مقصد را در استیت جدید فرستاده شده محسوب کنیم.
اضافه شدن استیت جدید در صورتی انجام میشود که در explored قرار نگرفته باشد.

از مزیتهای BFS میتوان به optimal بودن آن اشاره کرد.
 همچنین نسبت به IDS دارای زمان بهتری است.
 اما پیچیدگی فضایی آن نسبت به الگوریتمهای دیگر بیشتر است.


# تست 1:
|                       | فاصله جواب | تعداد استیت دیده شده | تعداد استیت مجزای دیده  شده | میانگین  زمان اجرا |
|-----------------------|------------|----------------------|-----------------------------|--------------------|
| ‌BFS                   | 20         | 256                  | 256                         | 0.009              |
| IDS                   | 20         | 2822                 | 172                         | 0.078              |
| A* (h1)               | 20         | 220                  | 220                         | 0.007              |
| A* (h2)               | 20         | 55                   | 55                          | 0.003              |
| A* (h2, $\alpha$ = 2) | 20         | 42                   | 42                          | 0.003              |
| A* (h2, $\alpha$ = 5) | 20         | 42                   | 42                          | 0.002              |

# تست ۲:
|                       | فاصله جواب | تعداد استیت دیده شده | تعداد استیت مجزای دیده  شده | میانگین  زمان اجرا |
|-----------------------|------------|----------------------|-----------------------------|--------------------|
| ‌BFS                   | 48         | 2304                 | 2304                        | 0.064              |
| IDS                   | 48         | 415316               | 2303                        | 12.643             |
| A* (h1)               | 48         | 402                  | 381                         | 0.013              |
| A* (h2)               | 48         | 402                  | 381                         | 0.003              |
| A* (h2, $\alpha$ = 2) | 48         | 120                  | 120                         | 0.004              |
| A* (h2, $\alpha$ = 5) | 48         | 120                  | 120                         | 0.003              |

# تست ۳:
|                     | فاصله جواب | تعداد استیت دیده شده | تعداد استیت مجزای دیده  شده | میانگین  زمان اجرا |
|---------------------|------------|----------------------|-----------------------------|--------------------|
| ‌BFS                 | 68         | 222935               | 222935                      | 5.364              |
| IDS                 | 68         | 4518330              | 216574                      | 125.374            |
| A* (h1)             | 68         | 142715               | 141844                      | 5.095              |
| A* (h2)             | 68         | 167                  | 167                         | 0.178              |
| A* (h2, $\alpha$ = 2) | 68         | 157                  | 157                         | 0.170              |
| A* (h2, $\alpha$ = 5) | 68         | 157                  | 157                         | 0.159              |

# تست ۴:
|                       | فاصله جواب | تعداد استیت دیده شده | تعداد استیت مجزای دیده  شده | میانگین  زمان اجرا |
|-----------------------|------------|----------------------|-----------------------------|--------------------|
| ‌BFS                   | 92         | 457                  | 457                         | 0.013              |
| IDS                   | 92         | 87774                | 316                         | 1.957              |
| A* (h1)               | 92         | 470                  | 456                         | 0.019              |
| A* (h2)               | 92         | 470                  | 456                         | 0.011              |
| A* (h2, $\alpha$ = 2) | 92         | 693                  | 441                         | 0.030              |
| A* (h2, $\alpha$ = 5) | 92         | 745                  | 428                         | 0.025              |